In [174]:
import numpy as np
from difflib import *
import os
import pandas as pd

In [175]:

import warnings
warnings.simplefilter('ignore')
# We will display plots right inside Jupyter Notebook
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import datetime

import re
import jieba
import mysql.connector
import matplotlib.font_manager as mfm

In [176]:
def get_connect():
    conn_temp = mysql.connector.connect(
            host = '139.162.122.115',#本地地址
            user = 'travel',#一般默认用户名
            passwd = 'ksd53890045',#本地数据库登录密码（这里用你自己的密码！！！）
            db = 'travelbigdata',#数据库名称（这里用你自己的数据库名称！！！）
            port = 3306,#安装mysql默认的端口号
            charset = 'utf8'#设置数据库统一编码  
    )
    return conn_temp

In [177]:
tablename='fb'
table={
    'id':'int(11) not null auto_increment PRIMARY KEY',
    'name':'varchar(100)',
    'rating':'float',
    'review':'float',
    'price':'varchar(100)',
    'url':'varchar(1024)',
    'types':'varchar(1024)',
    'checkin':'float',
    'follow':'float',
    'likes':'float',
    'recommend':'float',
    'phone':'varchar(100)',
    'vicinity':'varchar(1024)',
    'placeid':'varchar(100) ' ,
    'googlepk':'int(11)',
    'area':'varchar(100) ' ,
    'city':'varchar(100) ' ,
    'create_at':'datetime   NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT "創建時間"',
    'modify_at':' datetime NOT  NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT "修改時間"'
}
othersql='UNIQUE (placeid)'


In [178]:
def insetDB(tablename,table,data,conn_temp):
    cursor = conn_temp.cursor()
    sqltmpname=''
    sqltmpvalue=''
    for i in data:
        sqltmpname+=str(i)+' '+','
        if('char' in table[i]):
            sqltmpvalue+='"%s",' %data[i]
        else:
            sqltmpvalue+='%s,' %data[i]
    sqltmpname=sqltmpname[:-1]
    sqltmpvalue=sqltmpvalue[:-1]
    sqldata=[i for i in data]
    sqlInsert='insert into  '+tablename+'('+sqltmpname+') VALUES ('+sqltmpvalue+')'        
        
    
    sqlUpdate='UPDATE '+tablename+'  set rating=%s,review=%s where placeid= "%s" ' %\
        (data['rating'],data['review'],data['placeid'])
    
    
    try:
        cursor.execute(sqlInsert)
        conn_temp.commit()
    except:
        print('exsit   '+str(data['placeid']))
        cursor.execute(sqlUpdate)
        conn_temp.commit()

#    conn_temp.close()
    return 

In [179]:
foodtypes=['餐廳', '餐館', '咖啡', '熟食', '冰品', '飲料', '餃子', '牛扒', '果汁', 
      '早餐', '酒吧',  '拉麵', '居酒屋', '酒店', '雪糕', '甜品', '麵店', '麵包', 
      '冷麵', '麵食店', '美式扒房',       '熟食店', '珍珠奶茶', '包點', '壽司', 
      '烏冬專門店', '菜館', '三文治店', '薄餅', '曲奇餅店',  '酒樓', '海味店', 
      '餡餅店', '快餐', '酒家', '飲料店', '茶館', '餅店', '健康食品專賣店','燒烤', 
      '小食店','餐車店','餐廳','茶藝']
nottypes=['用品店','批發商']

In [148]:
###########################################
#
#
#
#
#
#
#
#
#########################################

In [180]:
##讀取資料
nameArea='Xinxing'
dataFB=pd.read_csv('data/FB/'+nameArea+'FB.csv')

In [181]:
## 讀取google 資料
dataGoogle=pd.read_csv('data/googleMerage/'+nameArea+'.csv')

In [182]:
### 定義一下高雄區域，利用地址分類資料  ######
kao=['鼓山區','三民區','新興區','前金區','苓雅區','鹽埕區','前鎮區','旗津區','左營區','鳳山區','大寮區','大樹區','鳥松區','小港區','楠梓區','林園區','岡山區','仁武區','大社區','旗山區']
place='高雄'

In [183]:
area=[]
city=[]
for st,i in enumerate(dataFB['addr']):
    areatmp=None
    placetmp=None
    for j in kao:
        if(str(j) in str(i)):
            areatmp=j
            placetmp=place
    area.append(areatmp)
    city.append(placetmp)
dataFB['area']=area
dataFB['city']=city

In [184]:
### 將FB的明子+上FB
name=[]
for i in dataFB.head().columns:
    name.append('FB'+i)
dataFB.columns=name

In [185]:
#### 判斷google 資料快速整理一下
def checkFood(onetype,types):
    boFood=False
    for t in types:
        if(t in onetype):
            boFood=True
            break
    return boFood

def checkNotFood(onetype,types):
    boFood=True
    for t in types:
        if(t in onetype):
            boFood=False
            break
    return boFood    
boCheck=[]
for i,j in zip(dataGoogle['googletype'],dataGoogle['type']):
    boTmp=False
    if( checkFood(str(i),foodtypes) and  checkNotFood(str(i),nottypes) ):
        boTmp=True
    if('food' in str(j)):
        boTmp=True
    boCheck.append(boTmp)

In [186]:
### 合併 google及 FB資料
datafood=dataGoogle[boCheck].reset_index(drop=True)
dataGoogleFB=pd.merge(datafood,dataFB,left_on='webUrl',right_on='FBwebUrl',how='right').reset_index(drop=True)

In [187]:
dataGoogleFB.columns

Index(['Unnamed: 0', 'from', 'icon', 'id', 'lat', 'lon', 'name', 'placeid',
       'price', 'rating_x', 'type', 'user_ratings_total', 'vicinity', 'cidUrl',
       'googletype', 'imageUrl', 'opentime', 'phone', 'rating_y', 'review',
       'webUrl', 'FBFBname', 'FBFBtypes', 'FBaddr', 'FBcheck', 'FBfollow',
       'FBlikes', 'FBphone', 'FBprice', 'FBrating', 'FBrecommend', 'FBreviwe',
       'FBwebUrl', 'FBarea', 'FBcity'],
      dtype='object')

In [188]:
conn_temp=get_connect()
for st in range(len(dataGoogleFB)):
    if(st<=-1):
        pass
    else:
        rating='NULL' if pd.isna(dataGoogleFB['FBrating'][st])else str(dataGoogleFB['FBrating'][st]).replace(',','')
        review='NULL' if pd.isna(dataGoogleFB['FBreviwe'][st])else str(dataGoogleFB['FBreviwe'][st]).replace(',','')
        checkin='NULL' if pd.isna(dataGoogleFB['FBcheck'][st])else str(dataGoogleFB['FBcheck'][st]).replace(',','')
        follow='NULL' if pd.isna(dataGoogleFB['FBfollow'][st])else str(dataGoogleFB['FBfollow'][st]).replace(',','')
        likes='NULL' if pd.isna(dataGoogleFB['FBlikes'][st])else   str(dataGoogleFB['FBlikes'][st]).replace(',','')
        recommend='NULL' if pd.isna(dataGoogleFB['FBrecommend'][st]) else   str(dataGoogleFB['FBrecommend'][st]).replace(',','')
        name=str(dataGoogleFB['FBFBname'][st]).replace(',','').replace('餐廳評論','').replace('Facebook','').replace('菜單','').replace('價格','').replace('|','').replace('、','')
        types=str(dataGoogleFB['FBFBtypes'][st]).replace(',','_')
        data={
            'name':name,
            'rating':rating,
            'review':review,
            'price':dataGoogleFB['FBprice'][st],
            'url':dataGoogleFB['FBwebUrl'][st],
            'types':types,
            'checkin':checkin,
            'follow':follow,
            'likes':likes,
            'recommend':recommend,
            'phone':dataGoogleFB['FBphone'][st],
            'vicinity':dataGoogleFB['FBaddr'][st],
            'placeid':dataGoogleFB['placeid'][st],
            'googlepk':'NULL',
            'area':dataGoogleFB['FBarea'][st],
            'city':dataGoogleFB['FBcity'][st]
        }
        insetDB('fb',table,data,conn_temp)
conn_temp.close()

exsit   ChIJPSQUUIgEbjQRxSHGqiQFhzE
exsit   ChIJ-0n3r4kEbjQRqpZlRR6Bip4
exsit   ChIJSzFnu4kEbjQRpx0ecoLQvak
exsit   ChIJkYQBsIkEbjQRR-1r73H3h3c
exsit   ChIJrU0_k4YEbjQR5SB6LXOWRdA
exsit   ChIJT1jZ84YEbjQRlBFjUNJJbl8
exsit   ChIJr94zFYYEbjQRn1rQ8Mc2e_4
exsit   ChIJc8rZto0EbjQR7T6vQgqWVtE
exsit   ChIJc8rZto0EbjQRvms5rD5ArJE
exsit   ChIJ84V57IkEbjQRmi8kNpeYxxc
exsit   ChIJXQ_IzWIEbjQRWq1ED9pVtb4
